In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import Paths
import TD_API
import api_keys
import time
import requests


In [2]:
# Get the most recent stock list file.
# list all files ends with .csv in given directory
files = [f for f in os.listdir(Paths.Stocks_List) if os.path.isfile(os.path.join(Paths.Stocks_List, f))]
files = [f for f in files if f.endswith('.csv')]
files.sort(reverse=True)
stock_list_df = pd.read_csv(os.path.join(Paths.Stocks_List, files[0])) 

In [3]:
stock_list =  stock_list_df['symbol'].tolist()
stock_list.sort()


In [4]:
def get_fundamental_update(symbol):
    #TD API limits request to 120 per second. That is maxium 2 request per second, so sleep here.
    time.sleep(0.5)
    #Define payload
    payload = {
            'apikey' : api_keys.APIKEY_FUNDAMENTAL,
            'symbol': symbol,
            'projection':'fundamental',
    }

    try:
        td_response = requests.get(url=TD_API.get_fundamental_endpoint(), params=payload)
        content = td_response.json()
        content_dict = content[symbol]['fundamental']
        df = pd.DataFrame(content_dict, index=[0])
        return df
    except Exception as ex:
        return ex

In [5]:
# The fundamental dataset should be requested every day. The original requested data will stored into folder, which named
# by the date. each stock will be merged to the production dataset which ready to use.

# Create a folder for today's data.
today = time.strftime("%Y-%m-%d")
updates_folder = os.path.join(Paths.Fundamental_Data_Update_Folder, today)
if not os.path.exists(updates_folder):
	os.makedirs(updates_folder)


In [7]:
"""
This script will try to get fundamental data for 4 times. 
During the process, the successed stock will be removed from the list, so next try will only on stocks that failed last time.
After 4 tries, the remaining stocks will be write to fail list file, stamped with date and stored in Report directory.
"""
for i in range(4):
	print('Try {}'.format(i))
	for symbol in stock_list:
		# print progress of the process
		print('Working on ' + symbol + ': {}/{}'.format(stock_list.index(symbol), len(stock_list)))
		try:
			df = get_fundamental_update(symbol)
			df.to_csv(os.path.join(updates_folder, symbol + '.csv'))
			# remove the stock from the list
			stock_list.remove(symbol)
		except Exception as ex:
			print(symbol + " failed")
	if len(stock_list) == 0:
		break

# If the stock list is not empty, write to fail list file in txt format.
if len(stock_list) > 0:
	with open(os.path.join(Paths.Fundamental_Data_Failed_List,  + today + '.txt'), 'w') as f:
		for symbol in stock_list:
			f.write(symbol + '\n')



Try 0
Working on AA: 0/14471
Working on AAC+: 1/14470
Working on AACIU: 2/14469
Working on AADR: 3/14468
Working on AAIN: 4/14467
Working on AAN: 5/14466
Working on AAON: 6/14465
Working on AAPL24: 7/14464
Working on AAPL29: 8/14463
Working on AAQC+: 9/14462
Working on AATC: 10/14461
Working on AAXJ: 11/14460
Working on ABAC: 12/14459
Working on ABBV21C: 13/14458
Working on ABBV27: 14/14457
Working on ABBV31: 15/14456
Working on ABCM: 16/14455
Working on ABG: 17/14454
Working on ABM: 18/14453
Working on ABP: 19/14452
Working on ABR-F: 20/14451
Working on ABSI: 21/14450
Working on ABST: 22/14449
Working on ABTX: 23/14448
Working on ABVC: 24/14447
Working on ABX: 25/14446
Working on AC.WD: 26/14445
Working on ACAD: 27/14444
Working on ACAHU: 28/14443
Working on ACAQ.U: 29/14442
Working on ACAw: 30/14441
ACAw failed
Working on ACB: 31/14441
Working on ACBAU: 32/14440
Working on ACBI: 33/14439
Working on ACCD: 34/14438
Working on ACDI.U: 35/14437
Working on ACEL: 36/14436
Working on ACES: 

In [ ]:
		percentage = round(100*(stock_list_length-len(stock_list))/stock_list_length)
		print('Working on ' + symbol + ': {}%'.format(percentage))